In [34]:
import numpy as np
from sklearn.linear_model import Lasso, Ridge
from sklearn.metrics import mean_squared_error
import pandas as pd
import matplotlib.pyplot as plt
from scipy.fft import fft
import pywt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [3]:
# 数据预处理
# 训练集
material_1 = pd.read_excel('F:/00程序/Githubclone_Projects/ICNN/datasets/CoreLosses/附件一（训练集）.xlsx', sheet_name='材料1')
material = material_1.iloc[:, :1068]
material

,温度，oC,频率，Hz,磁芯损耗，w/m3,励磁波形,0（磁通密度B，T）,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,25,50030,1997.955250,正弦波,0.000213,0.000389,0.000566,0.000743,0.000919,0.001096,...,-0.001551,-0.001374,-0.001198,-0.001022,-0.000846,-0.000669,-0.000492,-0.000316,-0.000140,0.000036
1,25,50020,2427.749830,正弦波,-0.000551,-0.000358,-0.000165,0.000028,0.000221,0.000413,...,-0.002476,-0.002284,-0.002091,-0.001899,-0.001707,-0.001515,-0.001322,-0.001129,-0.000937,-0.000744
2,25,50020,3332.725760,正弦波,-0.003780,-0.003564,-0.003349,-0.003134,-0.002919,-0.002704,...,-0.005927,-0.005713,-0.005499,-0.005285,-0.005070,-0.004855,-0.004640,-0.004425,-0.004210,-0.003995
3,25,50020,4502.908007,正弦波,-0.000511,-0.000267,-0.000023,0.000222,0.000466,0.000711,...,-0.002950,-0.002707,-0.002463,-0.002220,-0.001976,-0.001732,-0.001488,-0.001244,-0.001000,-0.000756
4,25,50030,6063.023248,正弦波,0.000458,0.000732,0.001007,0.001281,0.001555,0.001830,...,-0.002290,-0.002016,-0.001741,-0.001466,-0.001192,-0.000917,-0.000642,-0.000367,-0.000092,0.000183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3395,90,316460,25951.976130,梯形波,-0.014217,-0.013898,-0.013582,-0.013266,-0.012951,-0.012634,...,-0.016320,-0.016200,-0.016076,-0.015945,-0.015797,-0.015620,-0.015403,-0.015143,-0.014849,-0.014537
3396,90,316460,41673.753350,梯形波,-0.017910,-0.017508,-0.017108,-0.016711,-0.016312,-0.015912,...,-0.020542,-0.020390,-0.020234,-0.020069,-0.019883,-0.019663,-0.019394,-0.019070,-0.018703,-0.018312
3397,90,316460,53402.776280,梯形波,-0.020028,-0.019578,-0.019130,-0.018684,-0.018237,-0.017790,...,-0.023021,-0.022852,-0.022679,-0.022491,-0.022278,-0.022022,-0.021709,-0.021337,-0.020920,-0.020478
3398,90,316460,112528.009400,梯形波,-0.027652,-0.027021,-0.026392,-0.025764,-0.025138,-0.024513,...,-0.032188,-0.031942,-0.031679,-0.031381,-0.031029,-0.030602,-0.030094,-0.029522,-0.028912,-0.028285


In [28]:
data = material.iloc[1:, :]
T = material.iloc[:, 0]
f = material.iloc[:, 1]
P = material.iloc[:, 2]
B = material.iloc[:, 4:]
Bm = B.max(axis = 1)

0       0.028849
1       0.031419
2       0.035535
3       0.040015
4       0.045028
          ...   
3395    0.027540
3396    0.034780
3397    0.038987
3398    0.054829
3399    0.061444
Length: 3400, dtype: float64

In [31]:
# 假设 data 包含了所有的实验数据，包括 P、f、Bm、T
X = pd.concat([T, f, Bm], axis=1, ignore_index=True) # 特征矩阵
y = P

0         1997.955250
1         2427.749830
2         3332.725760
3         4502.908007
4         6063.023248
            ...      
3395     25951.976130
3396     41673.753350
3397     53402.776280
3398    112528.009400
3399    145226.508600
Name: 磁芯损耗，w/m3, Length: 3400, dtype: float64

In [32]:
# 将数据划分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
# 创建 Lasso 回归模型
lasso = Lasso(alpha=0.1)  

# 拟合 Lasso 回归模型
lasso.fit(X_train, y_train)

# 预测 Lasso 回归结果
y_pred_lasso = lasso.predict(X_test)

# 计算 Lasso 回归的均方误差
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
print(f"Lasso MSE: {mse_lasso}")

Lasso MSE: 48706034931.2613


In [36]:
# 创建岭回归模型
ridge = Ridge(alpha=0.1)  # 可以调整 alpha 参数

# 拟合岭回归模型
ridge.fit(X_train, y_train)

# 预测岭回归结果
y_pred_ridge = ridge.predict(X_test)

# 计算岭回归的均方误差
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print(f"Ridge MSE: {mse_ridge}")

Ridge MSE: 48661849718.666504
